In [19]:
import numpy as np

### Data I/O

In [20]:
data = open("shakespeare.txt", "r").read()
chars = list(set(data))
print(chars)
print(len(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

['c', 's', 'k', 'h', 'o', 'M', 'L', 'u', 't', 'd', 'b', 'j', 'E', 'g', ' ', 'C', 'A', 'U', 'G', ']', ':', 'l', 'W', '\n', '&', 'I', 'e', 'T', 'y', 'N', 'X', '[', "'", 'F', 'H', 'R', 'w', '-', 'x', 'r', 'O', 'K', '.', 'm', 'p', '$', 'a', ',', ';', 'V', 'i', '3', 'B', 'S', 'Z', 'q', 'n', 'f', 'P', 'Y', 'Q', 'v', '!', 'D', 'J', 'z', '?']
4573338


### Hyperparameter

In [21]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

### Model Parameter

In [22]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))
print(Wxh.shape)
print(Whh.shape)
print(Why.shape)

(100, 67)
(100, 100)
(67, 100)


In [23]:
p = 0
print(data[p:p+seq_length])
print(data[p+1:p+seq_length+1])
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
hprev = np.zeros((hidden_size, 1))
print(targets)
print(inputs)


First Citizen:
Before we 
irst Citizen:
Before we p
[50, 39, 1, 8, 14, 15, 50, 8, 50, 65, 26, 56, 20, 23, 52, 26, 57, 4, 39, 26, 14, 36, 26, 14, 44]
[33, 50, 39, 1, 8, 14, 15, 50, 8, 50, 65, 26, 56, 20, 23, 52, 26, 57, 4, 39, 26, 14, 36, 26, 14]


In [24]:
def lossFunction(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    loss = 0
    hs[-1] = np.copy(hprev)
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Whh, hs[t-1]) + np.dot(Wxh, xs[t]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0]) 
    return loss
        
        
loss = lossFunction(inputs, targets, hprev)     
print(loss)       

105.12318805251914


### Back Propagation

#### error = (Actual Output - Predicted Output)^2
#### delta w = learning rate * (de/dw)
#### W = w + delta w

In [30]:
p = 0
n = 0
X = []
Y = []
while (n < 500000):
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    X.append(inputs)
    Y.append(targets)
    p += seq_length
    n += 1 

In [31]:
print(len(X))
print(len(Y))

500000
500000


In [32]:
import re
with open('X.txt', 'w') as fx:
    for item in X:
        string = re.sub('[\[\]]','',repr(item))
        fx.write("%s\n" % string)

In [33]:
import re
with open('Y.txt', 'w') as fy:
    for item in Y:
        string = re.sub('[\[\]]','',repr(item))
        fy.write("%s\n" % string)